![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banner_Top_06.06.18.jpg?raw=true)

# Strathcona County Tree Location

##### Summary

This data set is provided by the Strathcona County, Environment https://data.strathcona.ca/Environment/Tree/v78i-7ntw

##### Content
Locations where there is a tree. The data are loaded 4 times per year from the 'treeworks' dataset. Last updated: April 6th, 2016.


## Downloading and parsing data into 'dataframe'

We begin by downloading the data directly from the website. 

https://data.strathcona.ca/Environment/Tree/v78i-7ntw

We can do this by selecting the 'API' tag and choosing CSV format on the top right side. Pressing the 'Copy' button will give us the URL we need to download the full dataset.


In [ ]:
!pip install git+https://github.com/python-visualization/folium -q

In [ ]:
# Import modules
# We will store the data into a 'dataframe' using pandas
import pandas as pd
# We want to be as precise as possible in keeping tree coordinates
from decimal import *
# We will visualize the coordinates in a map using the folium 
import folium
# We want to cluster them using the MarkerCluster submodule from folium plugins
from folium.plugins import MarkerCluster

print("Importing Python libraries was successful!")

In [ ]:
# Download data from API 
# Main Source: https://data.strathcona.ca/Environment/Tree/v78i-7ntw
# Pick API Tag - I chose the CSV option
link = "https://data.strathcona.ca/resource/v78i-7ntw.csv"

# Read and parse data as a pandas CSV
rawData = pd.read_csv(link)

# Rename columns
rawData = rawData.rename(columns={"treesiteid": "ID", "name": "name","location":"location"})

# Look at the first five columns
rawData.head()

---
### Exercise 1

Look at the table above. It has three columns: a tree id column (`ID`), a tree name column (`name`) and a tree location number (`location`).  

1. Look at the table above under the `location` column. Is there anything strange in the values under it? 

---

## Data Cleanup

The table contains an ID uniquely identifying each tree, a tree name and its location by using coordinates into a pair. 

A clean coordinate pair would look like

`(53.5227206433883, -113.324197520184)`

Our data set is not clean, we observe the presence of special characters `\n, \n` in each entry. We need to clean it up before we can visualize it. 

The special character `\n` is known as a 'line break'. 

This tells us that the coordinates are given as a string. 

In the cell below we will clean the data following three steps:

1. Remove special characters `\n, \n`
2. Remove left parenthesis `(` and right parenthesis `)`
3. Separate the pair into an `latitude` and `longitude` coordinates - and create separate columns (one for each)
4. Remove `'location'` column

In [ ]:
# Helper function to clean up data

def clean_dataframe(dataframe):
    try:
        """
        Function description: this function takes as input a dataframe with special characters
        and returns a 'clean' dataframe - a dataframe with no special characters and no parenthesis,
        as well as a latitude and longitude coordinate
        """
        
        # Remove special characters 
        dataframe = dataframe.replace('\n,  \n','',regex=True)
        # Data cleanup - Remove parenthesis 
        dataframe['location'] = dataframe['location'].str.replace('(','').str.replace(')','')

        dataframe[['latitude','longitude']] = dataframe['location'].str.split(",", n=1, expand=True)

        dataframe = dataframe.drop(['location'],axis=1)

        return dataframe
    
    except:
        
        print("WARNING! Make sure you are passing a pandas dataframe, and make sure your dataframe contains\
              a column named 'location' with comma-separated values.")

---
### Exercise 2

1. Use the helper function to cleanup the data. 
2. Run the cell and confirm that the odd characters and parenthesis were removed and that we have a latitude and a longitude column. 

---

In [ ]:
# Your code here
rawData = clean_dataframe(rawData)
# Look at the first five entries
rawData.head()

## Data Visualization

Now that we have cleaned up the dataframe and separated the string `location` values into separate numerical values containing the `X` and `Y` coordinates, we will use the `folium` package to visualize our data geographically. 


---
### Exercise 3 

1. Look up the coordinates for Strathcona County
2. In the cell below, enter the North coordinate (latitude) and the West coordinate (longitude) into separate variables (we have created the variable names for you). Make sure you enter numbers only!
3. These will be the initial coordinates that will help us locate our map. 
4. Run the cell to display the map. Ensure you are in the right location (hint: Edmonton should appear in the map)
---

In [ ]:
# Your code here 
latitude = 53.5701
longitude = -113.0741

# Initial coordinates 
SC_COORDINATES = [latitude, longitude]

# Create a map using our initial coordinates
map_osm=folium.Map(location=SC_COORDINATES, zoom_start=10, tiles='Stamen Terrain')

# Display the map 
display(map_osm)

## Displaying the tree locations

We can now add the tree locations into our map. 

In the cell below we will iterate over each record in our dataframe `rawData`. 

We will then add markers (one marker for each pair of coordinates) using the `folium.Marker` function. 

We will pass the `latitude` and `longitude` coordinates using the `location` parameter, and mark each tree with its `name` using the `popup` parameter. 

We will ad this to our `marker_cluster`, which has been added to our map `map_osm`. 

Run the cell below to see the locations of the trees.

In [ ]:
# Create marker cluster and add to our map
marker_cluster = MarkerCluster().add_to(map_osm)

# Iterate over each record, and add tree x and y coordinates, as well as tree name
MAX_RECORDS = len(rawData)
# For each record in rawData
for each in rawData[0:MAX_RECORDS].iterrows():
    # Use folium.Marker function, use X and Y coordinates to specify location
    folium.Marker(location = [each[1]['latitude'],each[1]['longitude']], 
                  # Add tree name
                  popup=folium.Popup(each[1]['name'],sticky=True),
                  #Make color/style changes here
                  icon=folium.Icon(color='green', icon='tree', prefix='fa'),
                  # Make sure our trees cluster nicely!
                  clustered_marker = True).add_to(marker_cluster)


# Show the map
display(map_osm)

# Optional - you can save this map as an HTML file
map_osm.save('TreeMap.html')

---
### Exercise 4

Use the interactive map above for this exercise. You will see 'clusters' of trees. 

Clusters with over 100 trees will be coloured in red, clusters with less than 100 trees will be coloured in yellow, while clusters with less than 10 trees will be coloured in green. A single tree has a green colour and a tree shape in it. 

1. Click on the largest cluster. It will break into smaller clusters. (Hint: it has over 900 trees)
2. How many red clusters do you see? How many yellow clusters? and green ones? (Hint: there are four red clusters)
3. Pick a red cluster and click on it. Are the clusters evenly distributed? If no, where are the clusters concentrated? 
4. Identify three areas that would benefit from populating with trees. What are the names of the streets/neighbourhoods where they are located?

#### Your observations here

---

## Further Visualization and Statistics

A natural question to ask is what is the most common kind of tree. 

We will next group and plot the data to find out which tree is the most common one. 

We start by setting up our visualizing environment. 

In [ ]:
#load "cufflinks" library under short name "cf"
import cufflinks as cf

#command to display graphics correctly in Jupyter notebook
cf.go_offline()

def enable_plotly_in_cell():
    import IPython
    from plotly.offline import init_notebook_mode
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
  '''))
    init_notebook_mode(connected=False)
    
get_ipython().events.register('pre_run_cell', enable_plotly_in_cell)

We are then going to group data by `name` using the `groupby()` method. 

We are then going to use the `size()` method to count how many trees of each kind there are. 

We will then sort the data. 

Run the cell below to show the five most common trees in Strathcona County. 

In [ ]:
# This cell groups trees by name, and counts them
count_by_tree_name = rawData.groupby("name").size().reset_index(name="count")
# once it does that, it sorts the counts in descending order
ordered_count = count_by_tree_name.sort_values(by='count', ascending=False)
# And displays the first 5 results. 
ordered_count.head()

We see that Green Ash is the most common tree in Strathcona County, followed by Spruce and Poplar. 

Let us visualize it. 

In [ ]:
ordered_count.iloc[0:5].iplot(kind="pie",values="count",labels="name",title="Five Most Common Trees") 

In [ ]:
ordered_count.iplot(kind="pie",values="count",labels="name",title="All Trees") 

---
### Exercise 5

1. Hover over the plots.
2. What is the percentage associated to each tree? 
3. What is the 'dominant' species? 
4. What tree species would you recommend restoring and why? 

#### Your observations here

---

## Final remarks

![alt text](https://github.com/callysto/callysto-sample-notebooks/blob/master/notebooks/images/Callysto_Notebook-Banners_Bottom_06.06.18.jpg?raw=true)